In [1]:
# Bakery Sales Prediction with Multiple Linear Regression
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# =============================================================================
# 1. LOAD DATA
# =============================================================================
print("=" * 80)
print("LOADING DATA")
print("=" * 80)

# Load datasets (update these paths to your actual file locations)
weather_path = "/workspaces/bakery_prediction/0_DataPreparation/wetter.csv"
test_path = "/workspaces/bakery_prediction/0_DataPreparation/test.csv"
train_path = "/workspaces/bakery_prediction/0_DataPreparation/train.csv"

weather = pd.read_csv(weather_path)
test = pd.read_csv(test_path)
train = pd.read_csv(train_path)

print(f"Weather data shape: {weather.shape}")
print(f"Train data shape: {train.shape}")
print(f"Test data shape: {test.shape}")

# Display date ranges
print(f"\nTraining data date range: {min(train['Datum'])} to {max(train['Datum'])}")
print(f"Test data date range: {min(test['Datum'])} to {max(test['Datum'])}")

# =============================================================================
# 2. MERGE DATASETS
# =============================================================================
print("\n" + "=" * 80)
print("MERGING DATASETS")
print("=" * 80)

# Merge weather data with train data (using all weather columns)
train_data = pd.merge(train, weather, on='Datum', how='left')
print(f"Merged training data shape: {train_data.shape}")

# Merge weather data with test data (using all weather columns or specific ones)
test_data = pd.merge(test, weather, on='Datum', how='left')
print(f"Merged test data shape: {test_data.shape}")

# =============================================================================
# 3. FILTER DATA BY DATE RANGES
# =============================================================================
print("\n" + "=" * 80)
print("FILTERING DATA BY DATE RANGES")
print("=" * 80)

# Define date thresholds
train_begin_date = "2013-07-01"
train_end_date = "2017-07-31"
validation_begin_date = "2017-08-01"
validation_end_date = "2018-07-31"
test_begin_date = "2018-08-01"
test_end_date = "2019-07-31"

# Filter training data
train_data = train_data[
    (train_data["Datum"] >= train_begin_date) & 
    (train_data["Datum"] <= train_end_date)
]

# Filter test data
test_data = test_data[
    (test_data["Datum"] >= test_begin_date) & 
    (test_data["Datum"] <= test_end_date)
]

print(f"Filtered training data shape: {train_data.shape}")
print(f"Filtered test data shape: {test_data.shape}")

# =============================================================================
# 4. HANDLE MISSING VALUES
# =============================================================================
print("\n" + "=" * 80)
print("HANDLING MISSING VALUES")
print("=" * 80)

print("\nMissing values in training data BEFORE cleaning:")
print(train_data.isna().sum())

print("\nMissing values in test data BEFORE cleaning:")
print(test_data.isna().sum())

# Drop rows with missing values in training data
train_data = train_data.dropna(how='any')

print(f"\nTraining data shape after dropping NaN: {train_data.shape}")

# For test data, you might want to fill missing values instead of dropping
# since you need predictions for all test rows
# Option 1: Fill with mean
test_data['Windgeschwindigkeit'] = test_data['Windgeschwindigkeit'].fillna(
    test_data['Windgeschwindigkeit'].mean()
)

# If there are other columns with missing values, handle them too
# For example, fill numeric columns with mean
numeric_cols = test_data.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if test_data[col].isna().sum() > 0:
        test_data[col] = test_data[col].fillna(test_data[col].mean())

print("\nMissing values in test data AFTER cleaning:")
print(test_data.isna().sum())

# =============================================================================
# 5. PREPARE FEATURES AND TARGET
# =============================================================================
print("\n" + "=" * 80)
print("PREPARING FEATURES FOR MODEL")
print("=" * 80)

# Define independent variables (features) - NOW INCLUDING BOTH VARIABLES
X_train = train_data[["Warengruppe", "Windgeschwindigkeit"]]
y_train = train_data["Umsatz"]

print(f"Training features shape: {X_train.shape}")
print(f"Training target shape: {y_train.shape}")
print(f"\nFeature statistics:")
print(X_train.describe())

# Add constant term for intercept
X_train_with_const = sm.add_constant(X_train)

# =============================================================================
# 6. TRAIN THE MODEL
# =============================================================================
print("\n" + "=" * 80)
print("TRAINING MULTIPLE LINEAR REGRESSION MODEL")
print("=" * 80)

# Train OLS model with BOTH independent variables
model = sm.OLS(y_train, X_train_with_const).fit()

# Display model summary
print(model.summary())

# Extract coefficients
print("\n" + "=" * 80)
print("MODEL COEFFICIENTS")
print("=" * 80)
print(f"Intercept: {model.params['const']:.4f}")
print(f"Coefficient for Warengruppe: {model.params['Warengruppe']:.4f}")
print(f"Coefficient for Windgeschwindigkeit: {model.params['Windgeschwindigkeit']:.4f}")

print(f"\nRegression Equation:")
print(f"Umsatz = {model.params['const']:.4f} + "
      f"{model.params['Warengruppe']:.4f}*Warengruppe + "
      f"{model.params['Windgeschwindigkeit']:.4f}*Windgeschwindigkeit")

# =============================================================================
# 7. MAKE PREDICTIONS ON TRAINING DATA (for evaluation)
# =============================================================================
train_predictions = model.predict(X_train_with_const)

# Calculate training metrics
train_mae = mean_absolute_error(y_train, train_predictions)
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
train_r2 = r2_score(y_train, train_predictions)
train_mape = (abs((y_train - train_predictions) / y_train).mean()) * 100

print("\n" + "=" * 80)
print("TRAINING SET PERFORMANCE")
print("=" * 80)
print(f"R² Score: {train_r2:.4f}")
print(f"Mean Absolute Error (MAE): {train_mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {train_rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {train_mape:.2f}%")

# =============================================================================
# 8. MAKE PREDICTIONS ON TEST DATA
# =============================================================================
print("\n" + "=" * 80)
print("MAKING PREDICTIONS ON TEST DATA")
print("=" * 80)

# Prepare test features - SAME FEATURES AS TRAINING
X_test = test_data[["Warengruppe", "Windgeschwindigkeit"]]
X_test_with_const = sm.add_constant(X_test)

# Make predictions
predictions = model.predict(X_test_with_const)

print(f"Number of predictions: {len(predictions)}")
print(f"Sample predictions (first 10):")
print(predictions.head(10))

# =============================================================================
# 9. CREATE SUBMISSION FILE
# =============================================================================
print("\n" + "=" * 80)
print("CREATING SUBMISSION FILE")
print("=" * 80)

# Create predictions dataframe
pred_df = pd.DataFrame({
    'id': test_data['id'],
    'Umsatz': predictions
})

print(f"Prediction dataframe shape: {pred_df.shape}")
print(f"\nFirst 10 predictions:")
print(pred_df.head(10))

# Save to CSV
output_path = "/workspaces/bakery_prediction/1_DatasetCharacteristics/predictions.csv"
pred_df.to_csv(output_path, index=False)
print(f"\nPredictions saved to: {output_path}")

# =============================================================================
# 10. VISUALIZATION
# =============================================================================
print("\n" + "=" * 80)
print("CREATING VISUALIZATIONS")
print("=" * 80)

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Plot 1: Actual vs Predicted (Training)
axes[0, 0].scatter(y_train, train_predictions, alpha=0.5)
axes[0, 0].plot([y_train.min(), y_train.max()], 
                [y_train.min(), y_train.max()], 'r--', lw=2)
axes[0, 0].set_xlabel('Actual Umsatz')
axes[0, 0].set_ylabel('Predicted Umsatz')
axes[0, 0].set_title(f'Training: Actual vs Predicted\nR² = {train_r2:.4f}')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Residuals
residuals = y_train - train_predictions
axes[0, 1].scatter(train_predictions, residuals, alpha=0.5)
axes[0, 1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[0, 1].set_xlabel('Predicted Umsatz')
axes[0, 1].set_ylabel('Residuals')
axes[0, 1].set_title('Residual Plot')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Residual Distribution
axes[0, 2].hist(residuals, bins=50, alpha=0.7, edgecolor='black')
axes[0, 2].set_xlabel('Residuals')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].set_title('Distribution of Residuals')
axes[0, 2].grid(True, alpha=0.3)

# Plot 4: Umsatz vs Warengruppe
axes[1, 0].scatter(train_data['Warengruppe'], y_train, alpha=0.3)
axes[1, 0].set_xlabel('Warengruppe')
axes[1, 0].set_ylabel('Umsatz')
axes[1, 0].set_title('Umsatz vs Warengruppe')
axes[1, 0].grid(True, alpha=0.3)

# Plot 5: Umsatz vs Windgeschwindigkeit
axes[1, 1].scatter(train_data['Windgeschwindigkeit'], y_train, alpha=0.3)
axes[1, 1].set_xlabel('Windgeschwindigkeit')
axes[1, 1].set_ylabel('Umsatz')
axes[1, 1].set_title('Umsatz vs Windgeschwindigkeit')
axes[1, 1].grid(True, alpha=0.3)

# Plot 6: Q-Q Plot for normality of residuals
stats.probplot(residuals, dist="norm", plot=axes[1, 2])
axes[1, 2].set_title('Q-Q Plot (Normality of Residuals)')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
viz_path = "/workspaces/bakery_prediction/1_DatasetCharacteristics/model_diagnostics.png"
plt.savefig(viz_path, dpi=300, bbox_inches='tight')
print(f"Visualizations saved to: {viz_path}")

print("\n" + "=" * 80)
print("ANALYSIS COMPLETE!")
print("=" * 80)

ModuleNotFoundError: No module named 'pandas'